In [ ]:
import torch.nn as nn
import torch
import numpy as np

### Classification classes number setting

In [ ]:
# Can change this later when updated labels
number_classes = 2

### Choosing the device

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

### Each path to csv


In [8]:
data = 'point_history.csv'
datast_label = 'point_history_classifier_label.csv'

### Reading the csv files and loading the learning data


https://www.w3schools.com/python/python_file_open.asp

In [9]:
X_data = []
y_data = []

with open(data, 'r') as file:
  for line in file:
    values = line.strip().split(',')
    label = int(values[0])  # First element as label
    features = [float(v) for v in values[1:]]  # Remaining as features
    y_data.append(label)
    X_data.append(features)
    print(f"Label: {label}, Features: {features}")

Label: 3, Features: [0.0, 0.0, -4.930773284286261e-05, -1.874847544564141e-05, -6.677827332168818e-05, 0.00029736442698372734]
Label: 3, Features: [0.0, 0.0, -4.930773284286261e-05, -1.874847544564141e-05, -6.677827332168818e-05, 0.00029736442698372734, 4.8206443898379804e-05, 0.0005504603187243144, 6.096507422626018e-06, 0.0006631281640794543, -5.042441189289093e-05, 0.0006911898652712504]
Label: 3, Features: [0.0, 0.0, -4.930773284286261e-05, -1.874847544564141e-05, -6.677827332168818e-05, 0.00029736442698372734]
Label: 3, Features: [0.0, 0.0, -4.930773284286261e-05, -1.874847544564141e-05, -6.677827332168818e-05, 0.00029736442698372734, 4.8206443898379804e-05, 0.0005504603187243144, 6.096507422626018e-06, 0.0006631281640794543, -5.042441189289093e-05, 0.0006911898652712504]
Label: 3, Features: [0.0, 0.0, -4.930773284286261e-05, -1.874847544564141e-05, -6.677827332168818e-05, 0.00029736442698372734, 4.8206443898379804e-05, 0.0005504603187243144, 6.096507422626018e-06, 0.0006631281640

### Convert lists to PyTorch tensors

https://discuss.pytorch.org/t/best-way-to-convert-a-list-to-a-tensor/59949

In [10]:
# Finding the maximum number of features
max_features = len(X_data[0])
for feature in X_data:
  max_features = max(max_features, len(feature))


# Pad shorter feature lists with zeros for consistent tensor shape
for features in X_data:
    features += [0.0] * (max_features - len(features))




X_tensor = torch.tensor(X_data, dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.long)
X_tensor

tensor([[ 0.0000e+00,  0.0000e+00, -4.9308e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -4.9308e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -4.9308e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00, -5.0246e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -5.0246e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -5.0246e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

### Feed forward model


In [11]:
class Feedforward(nn.Module):

    def __init__(self, input_size, num_classes):

      super().__init__()

      # nn.Sequential is a container that allows to build neural networks
      # in a sequential, layer-by-layer format.
      self.ff = nn.Sequential (
          # I'm guessing the number of inputs of labels?
          #
          # Have 4 labels for now Default, Moving Cursor, Scroll Down, Scroll Up
          nn.Linear(input_size, 20), # Input layer with the max length of the feature list, intermediate with 20 nodes
          nn.ReLU(), # Activation function
          nn.Linear(100, 100), # Intermediate layer with 100 nodes
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, num_classes), # Output layer with `num_classes` nodes
          nn.Softmax(dim=1)  # Softmax for probabilities
      )

    def forward(self, x):
        return self.ff(x)

### Insatantiate the model😈

In [12]:
model = Feedforward(input_size=max_features, num_classes=number_classes).to(device)

In [13]:
# Saving the model
def save_model(model, path='feedforward_model.pth'):
    torch.save(model.state_dict(), path)  # Save model parameters to a file

# Loading the model
def load_model(model, path='feedforward_model.pth'):
    model.load_state_dict(torch.load(path))  # Load parameters into the model
    print(model.eval())  # Set model to evaluation mode (useful for inference)
    print("Model loaded successfully.")

# Usage Example
save_model(model)  # Save the model
# Later, to reload:
load_model(model)


Feedforward(
  (ff): Sequential(
    (0): Linear(in_features=78, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=2, bias=True)
    (11): Softmax(dim=1)
  )
)
Model loaded successfully.


<ipython-input-13-66018aba7929>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))  # Load parameters into the model
